In [54]:

import numpy as np
import networkx as nx
import hashlib
G = nx.read_edgelist("large_twitch_edges.csv", delimiter=',')


In [55]:
print(G)

Graph with 168114 nodes and 6797557 edges


In [56]:
def DegreeDiscountHeuristic(seed_num,G, p):
    degree_dict = {}

    for tuple in G.degree():
        degree_dict[tuple[0]] = tuple[1]
    seeds = []
    for i in range(seed_num):
        max_degree = max(degree_dict, key=degree_dict.get)
        seeds.append(max_degree)
        degree_dict.pop(max_degree)
        for node in G.neighbors(max_degree):
            if node in degree_dict:
                degree_dict[node] = degree_dict[node] - 2*(i+1) - ( degree_dict[node] - i - 1)*(i+1)*p
    return seeds
            

In [57]:
def DegreeHeuristic(seed_num,G):
    degree_heuristic = [node for (node, val) in sorted(G.degree(), reverse=True,key=lambda pair: pair[1])]
    seeds = degree_heuristic[0:seed_num]
    return seeds

In [58]:
def LIRHeuristic(seed_num,G):
    degree_dict = {}

    for tuple in G.degree():
        degree_dict[tuple[0]] = tuple[1]
    seeds = []
    LIR_0 = []
    for i in range(seed_num):
        for node in degree_dict:
            LIR_value = 0
            for neighbor in G.neighbors(node):
                if degree_dict[node] < degree_dict[neighbor]:
                    LIR_value = LIR_value + 1
                    if LIR_value > i:
                        break
            if LIR_value == i:
                LIR_0.append([node, degree_dict[node], LIR_value])
        if len(LIR_0) >= seed_num:
                break
            
    LIR_heuristic = [(node) for (node, val, lir) in sorted(sorted(LIR_0, reverse=True,key=lambda pair: pair[1]), key=lambda pair: pair[2])]
    return LIR_heuristic[0:seed_num]


In [59]:
def IndependentCascade(G, seeds, p, perm_seed):
    np.random.seed(perm_seed)
    active_nodes = seeds
    spread = []
    while len(active_nodes) > 0:
        curr  = active_nodes[0]
        for  neighbor in G.neighbors(curr):
            if neighbor not in active_nodes and neighbor not in spread:
                if np.random.uniform(0,1) <= p:
                    active_nodes.append(neighbor)
        spread.append(curr)
        active_nodes.remove(active_nodes[0])
    return len(spread)


In [ ]:
seed_range = 100
p = 0.002
permutation_num = 100
perm_seeds = []
seed = 1000
for i in range(permutation_num):
    perm_seeds.append(int(hashlib.sha256(str(seed+i).encode()).hexdigest(),16)%4294967295)
DDH_averages = []
Degree_averages = []
LIR_averages = []
for i in range(seed_range):
    seeds_DDH = DegreeDiscountHeuristic(i,G,p)
    seeds_Degree = DegreeHeuristic(i,G)
    seeds_LIR = LIRHeuristic(i,G)
    seed_count_DDH = 0
    seed_count_Degree = 0
    seed_count_LIR = 0
    for j in range(permutation_num):
        seed_count_DDH = seed_count_DDH + IndependentCascade(G, seeds_DDH, p,perm_seeds[j])
        seed_count_Degree = seed_count_Degree + IndependentCascade(G, seeds_Degree, p,perm_seeds[j])
        seed_count_LIR = seed_count_LIR + IndependentCascade(G, seeds_LIR, p,perm_seeds[j])
    DDH_averages.append(int(seed_count_DDH/permutation_num))
    Degree_averages.append(int(seed_count_Degree/permutation_num))
    LIR_averages.append(int(seed_count_LIR/permutation_num))
print(LIR_averages)

[0, 18, 21, 16, 20, 24, 21, 21, 24, 18, 19, 14, 21, 20, 21, 22, 20, 23, 19, 14, 18, 17, 24, 14, 12, 18, 21, 19, 21, 25, 17, 25, 21, 21, 25, 22, 26, 26, 26, 19, 21, 17, 18, 22, 24, 26, 26, 26, 25, 19, 22, 22, 16, 19, 16, 15, 15, 20, 19, 17, 21, 21, 19, 24, 19, 20, 23, 22, 22, 14, 14, 18, 25, 25, 25, 25, 19, 16, 17, 20, 18, 18, 21, 21, 21, 24, 20, 15, 22, 17, 23, 23, 25, 23, 23, 20, 23, 22, 22, 25]


In [63]:
print(DDH_averages)
print(Degree_averages)
print(LIR_averages)

[0, 18, 22, 19, 17, 21, 21, 23, 26, 25, 19, 21, 18, 20, 25, 26, 27, 25, 22, 26, 22, 25, 22, 26, 24, 26, 24, 26, 27, 24, 22, 25, 26, 24, 28, 25, 28, 27, 26, 27, 30, 28, 30, 27, 28, 29, 30, 26, 30, 29, 29, 30, 30, 29, 30, 28, 29, 31, 29, 29, 30, 32, 30, 30, 29, 28, 30, 29, 31, 30, 29, 31, 30, 32, 32, 32, 33, 31, 32, 32, 30, 34, 32, 30, 31, 34, 32, 34, 32, 33, 34, 34, 31, 31, 34, 33, 32, 33, 33, 35]
[0, 18, 22, 19, 17, 21, 21, 23, 26, 25, 19, 21, 18, 20, 25, 26, 27, 25, 22, 26, 22, 25, 22, 26, 24, 26, 24, 26, 27, 24, 22, 25, 26, 24, 28, 25, 28, 27, 26, 27, 30, 28, 30, 27, 28, 29, 30, 26, 30, 29, 29, 30, 30, 29, 30, 28, 29, 31, 29, 29, 30, 32, 30, 30, 29, 28, 30, 29, 31, 30, 29, 31, 30, 32, 32, 32, 33, 31, 32, 32, 30, 34, 32, 30, 31, 34, 32, 34, 32, 33, 34, 34, 31, 31, 34, 33, 32, 33, 33, 35]
[0, 18, 21, 16, 20, 24, 21, 21, 24, 18, 19, 14, 21, 20, 21, 22, 20, 23, 19, 14, 18, 17, 24, 14, 12, 18, 21, 19, 21, 25, 17, 25, 21, 21, 25, 22, 26, 26, 26, 19, 21, 17, 18, 22, 24, 26, 26, 26, 25, 19, 